In [1]:
import numpy as np
import pandas as pd

training_data = pd.read_csv('input/training_data.csv')
training_data.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1
0,27,17,2,1.0,19,2014,5,0.0,0.0,0.0,0.0
1,30,15,2,1.0,40,2014,3,0.0,0.0,0.0,1.0
2,30,16,2,1.0,40,2014,4,1.0,0.0,0.0,0.0
3,31,16,2,1.0,37,2014,4,0.0,0.0,0.0,0.0
4,32,12,2,1.0,40,2014,0,0.0,0.0,0.0,0.0


In [2]:
from sklearn.preprocessing import StandardScaler

cnt_scaler = StandardScaler()
cnt_scaler.fit(training_data.item_cnt_tp1.values.reshape(-1, 1))
training_data.item_cnt_tp1 = cnt_scaler.transform(training_data.item_cnt_tp1.values.reshape(-1, 1))
training_data.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1
0,27,17,2,1.0,19,2014,5,0.0,0.0,0.0,-0.151330
1,30,15,2,1.0,40,2014,3,0.0,0.0,0.0,-0.033937
2,30,16,2,1.0,40,2014,4,1.0,0.0,0.0,-0.151330
3,31,16,2,1.0,37,2014,4,0.0,0.0,0.0,-0.151330
4,32,12,2,1.0,40,2014,0,0.0,0.0,0.0,-0.151330


In [3]:
y = training_data['item_cnt_tp1']
X = training_data.drop(['date_block_num','item_cnt_tp1'], axis=1)
X.head()

,item_id,shop_id,item_cnt_day,item_category_id,year,month,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12
0,27,2,1.0,19,2014,5,0.0,0.0,0.0
1,30,2,1.0,40,2014,3,0.0,0.0,0.0
2,30,2,1.0,40,2014,4,1.0,0.0,0.0
3,31,2,1.0,37,2014,4,0.0,0.0,0.0
4,32,2,1.0,40,2014,0,0.0,0.0,0.0


In [4]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 42)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Training set has {} columns.".format(X_train.shape[1]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 701061 samples.
Training set has 9 columns.
Testing set has 175266 samples.


In [5]:
from sklearn.metrics import mean_squared_error

def calc_RMSE(actuals, predictions):
    return np.sqrt(mean_squared_error(actuals, predictions))

In [6]:
y_train_orig = cnt_scaler.inverse_transform(y_train)
y_test_orig = cnt_scaler.inverse_transform(y_test)

def assess_model(neuralnet):
    predictions_train = cnt_scaler.inverse_transform(neuralnet.predict(X_train.values))
    print('Training RMSE:', calc_RMSE(y_train_orig, predictions_train))

    predictions_test = cnt_scaler.inverse_transform(neuralnet.predict(X_test.values))
    print('Testing RMSE:', calc_RMSE(y_test_orig, predictions_test))

In [7]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(32, input_dim=9))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(32, input_dim=9))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1, activation='relu'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
model.summary()

model.fit(X_train.values, y_train.values, batch_size=128, epochs=20)

/home/dylbert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
__________

In [8]:
assess_model(model)

Training RMSE: 11.343940824636643
Testing RMSE: 10.576288605256075
